In [150]:
import re
import itertools
from dataclasses import dataclass
from more_itertools import chunked
from collections import defaultdict
from pprint import pprint

In [151]:
@dataclass
class Pattern:
    name: str
    pattern: bytes

In [152]:
def hex_to_bytes(s: str) -> bytes:
    return int(s, 16).to_bytes(1, "little")

In [153]:
def convert_to_pattern(s: list[str]) -> list[int | None]:
    pattern = [None if item == "??" else int(item, 16) for item in s]
    return pattern

In [154]:
def check_pattern(buffer: bytes, start_index: int, pattern: list[int | None]) -> bool:
    for i, c in enumerate(pattern):
        if c is None:
            continue
            
        if buffer[start_index + i] != pattern[i]:
            return False
    
    return True

In [155]:
patterns_dict = defaultdict(list)
patterns = []

with open("fn_byte_patterns.ffsess") as patterns_file:
    for tab_line, pattern_line in chunked(patterns_file, 2):
        command, _, tab_name = tab_line.rstrip().partition(" ")
        assert command == "Tab", command
        rule_name, _, _, *pattern = pattern_line.rstrip().split(" ")
        assert rule_name == "RuleBytePattern"
        pattern = convert_to_pattern(pattern)
        pattern_object = Pattern(tab_name, pattern)
        patterns_dict[pattern[0]].append(pattern_object)
        patterns.append(pattern_object)

In [156]:
patterns_dict

defaultdict(list,
            {72: [Pattern(name='string_copy', pattern=[72, 137, 81, None, 72, 131, 121, None, None, 114, None, 72, 139, 1, 198, 4, 16, None, 195, 198, 4, 17, None, 195]),
              Pattern(name='string_copy_n', pattern=[72, 137, 92, 36, None, 72, 137, 108, 36, None, 86, 87, 65, 87, 72, 131, 236, None, 72, 139, 105, None, 73, 139, 240, 76, 139, 250, 72, 139, 217, 76, 59, 197, 119, None, 72, 139, 249, 72, 131, 253, None, 114, None, 72, 139, 57, 72, 137, 113, None, 72, 139, 207, 232, None, None, None, None, 198, 4, 55, None, 233, None, None, None, None, 72, 191, None, None, None, None, None, None, None, None, 72, 59, 247, 15, 135, None, None, None, None, 72, 139, 206, 76, 137, 116, 36, None, 72, 131, 201, None, 72, 59, 207, 119, None, 72, 139, 213, 72, 139, 199, 72, 209, 234, 72, 43, 194, 72, 59, 232, 119, None, 72, 141, 4, 42, 72, 139, 249, 72, 59, 200, 72, 15, 66, 248, 72, 141, 79, None, 232, None, None, None, None, 76, 139, 198, 72, 137, 115, None, 73, 139, 215, 7

In [157]:
path = "/mnt/second/SteamLibrary/steamapps/common/Dwarf Fortress/Dwarf Fortress.exe"

In [170]:
found = defaultdict(list)

In [171]:
with open(path, "rb") as file:
    data = file.read()
    for i, c in enumerate(data):
        possible_patterns = patterns_dict.get(c)
        if possible_patterns:
            for pattern in possible_patterns:
                if check_pattern(data, i, pattern.pattern):
                    found[pattern.name].append(i)

In [174]:
diff_to_rva = 0xc00

In [175]:
for pattern in patterns:
    for offset in found[pattern.name]:
        print(f"{pattern.name} = 0x{offset + diff_to_rva:X}")

string_copy = 0xB830
string_copy_n = 0xB850
string_append = 0xB5F0
string_append = 0xB610
string_append_0 = 0xB5F0
string_append_0 = 0xB610
string_append_n = 0xB990
convert_ulong_to_string = 0x393810
addst = 0x7C5220
addst_top = 0x7C5300
addcoloredst = 0x7C4ED0
addst_flag = 0x7C4FB0
standartstringentry = 0x8C3F80
simplify_string = 0x394350
upper_case_string = 0x394690
lower_case_string = 0x3944F0
capitalize_string_words = 0x394830
capitalize_string_first_word = 0x394AB0
addchar = 0x561E0
addchar = 0xF3D10
addchar_top = 0x561E0
addchar_top = 0xF3D10
addtexture = 0xEDBB20
gps_allocate = 0x5FF3C0
cleanup_arrays = 0x5FF1E0
screen_to_texid = 0x5F7450
screen_to_texid_top = 0x5F7640
loading_world_new_game_loop = 0xA4B4D0
loading_world_continuing_game_loop = 0x5A1810
loading_world_start_new_game_loop = 0x59F460
menu_interface_loop_steam = 0x173100
